<a href="https://www.kaggle.com/code/ahmedrushdi/sarcasm?scriptVersionId=126865138" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np 
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json
/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json


In [2]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize


json_file_path1 = '/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json'
json_file_path2 = '/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json'


# Read the JSON files using pandas
data1 = pd.read_json(json_file_path1, lines=True)
data2 = pd.read_json(json_file_path2, lines=True)



# Combine the data into a single DataFrame
data = pd.concat([data1, data2])

data = data.drop('article_link', axis = 1).drop_duplicates()
data

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0
...,...,...
28497,tyson holds contest to let fans submit new ide...,1
28509,increasingly cocky bernie sanders announces he...,1
28520,cash-strapped zuckerberg forced to sell 11 mil...,1
28544,grocery store bar actually has great little ha...,1


In [3]:
data.groupby('is_sarcastic')

In [4]:
corpus = data['headline']
labels = data['is_sarcastic']

In [5]:
import tensorflow as tf
import gensim.downloader as api
import numpy as np
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.layers import Embedding, LSTM, Dense, GRU, Dropout, Bidirectional

In [6]:
w2v_model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [7]:
vocab_size = len(w2v_model.key_to_index) +2  # Add 1 for the padding token
embedding_dim = w2v_model.vector_size

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, idx in w2v_model.key_to_index.items():
    embedding_matrix[idx+2] = w2v_model[word]

embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    trainable=False,
)

In [8]:
len(list(w2v_model.key_to_index.keys()))

400000

In [9]:
input_dim = vocab_size + 2  # Size of the vocabulary
input_length = 200  # Length of input sequences

text_vectorization = TextVectorization(
    max_tokens=input_dim,
    output_mode="int",
    output_sequence_length=input_length,
    vocabulary = list(w2v_model.key_to_index.keys())
)
# text_vectorization.set_vocabulary(['<UNK>']+list(w2v_model.key_to_index.keys()))

In [10]:
# Sanity check
w2v_model['the'] - embedding_layer(text_vectorization([['the']]))[0,0]

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>

In [11]:
model = tf.keras.Sequential([
    text_vectorization,  # TextVectorization layer
    embedding_layer,  # Embedding layer
    Bidirectional(LSTM(64, return_sequences = False), input_shape=(input_length, embedding_dim)),
    Dense(1, activation='sigmoid'),   # Output
])

In [12]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(corpus, labels, epochs=20, batch_size=64, validation_split=0.2)

Epoch 1/20
357/357 [==============================] - 204s 520ms/step - loss: 0.5426 - accuracy: 0.7272 - val_loss: 0.5320 - val_accuracy: 0.7369
Epoch 2/20
357/357 [==============================] - 181s 506ms/step - loss: 0.4309 - accuracy: 0.8017 - val_loss: 0.4475 - val_accuracy: 0.7900
Epoch 3/20
357/357 [==============================] - 181s 508ms/step - loss: 0.3809 - accuracy: 0.8295 - val_loss: 0.4140 - val_accuracy: 0.8079
Epoch 4/20
357/357 [==============================] - 182s 510ms/step - loss: 0.3472 - accuracy: 0.8465 - val_loss: 0.4353 - val_accuracy: 0.8002
Epoch 5/20
357/357 [==============================] - 180s 505ms/step - loss: 0.3208 - accuracy: 0.8605 - val_loss: 0.4232 - val_accuracy: 0.8092
Epoch 6/20
357/357 [==============================] - 181s 507ms/step - loss: 0.2961 - accuracy: 0.8717 - val_loss: 0.4199 - val_accuracy: 0.8153
Epoch 7/20
357/357 [==============================] - 183s 512ms/step - loss: 0.2755 - accuracy: 0.8820 - val_loss: 0.3529 -

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 200)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 200, 50)           20000100  
                                                                 
 bidirectional (Bidirectiona  (None, 128)              58880     
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 20,059,109
Trainable params: 59,009
Non-trainable params: 20,000,100
_________________________________________________________________


In [14]:
model(corpus[:10])

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[3.0726843e-04],
       [2.4761061e-05],
       [9.9224949e-01],
       [8.7906945e-01],
       [2.8137568e-02],
       [1.6581507e-04],
       [7.2690159e-02],
       [7.2174251e-02],
       [9.8642820e-01],
       [9.0110898e-03]], dtype=float32)>

In [15]:
labels[:10]

0    0
1    0
2    1
3    1
4    0
5    0
6    0
7    0
8    1
9    0
Name: is_sarcastic, dtype: int64